In [ ]:
# Cell 1: Imports and Global Setup
import os
import random
from pathlib import Path
import re
from itertools import groupby

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import integrate
from tqdm import trange, tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader


In [ ]:
# 1. 장치 설정 (CUDA 또는 CPU)
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {DEVICE}")

# 2. 결과 재현성을 위한 시드 고정
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed) # if use multi-GPU
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)

# 3. 정규화를 위한 전역 변수 초기화
maxV_global = 0
Q_limited_list_max_global = 0
Cycle_count_max_global = 2200 # 초기값 설정

In [ ]:
# Cell 2: Core Feature Extraction Function (dQdV)
def dQdV(file, q_rated, plot_data=False):
    """
    배터리 사이클 데이터 파일로부터 dQ/dV 특징을 추출합니다.
    
    Args:
        file (str): CSV 파일 경로
        q_rated (int): 배터리의 정격 용량 (mAh)
        plot_data (bool): 데이터 확인을 위한 그래프를 그릴지 여부
        
    Returns:
        tuple: (maxV, Q_limited_list, Cycle_count) 시계열 데이터
    """
    d1 = pd.read_csv(file)[["Cycle_Number", "Voltage_V", "Current_mA", "Time_s", "Capacity_mAh"]]
    
    # 요청하신 데이터 시각화 부분
    if plot_data:
        print(f"--- 데이터 확인: {Path(file).name} ---")
        plt.figure(figsize=(12, 6))
        plt.suptitle(f'Overall Data for {Path(file).name}', fontsize=16)
        
        plt.subplot(2, 1, 1)
        plt.plot(d1.index, d1['Voltage_V'], label='Voltage (V)')
        plt.title('Voltage over all cycles')
        plt.xlabel('Data Point Index'); plt.ylabel('Voltage (V)'); plt.grid(True); plt.legend()
        
        plt.subplot(2, 1, 2)
        plt.plot(d1.index, d1['Current_mA'], label='Current (mA)', color='orange')
        plt.title('Current over all cycles')
        plt.xlabel('Data Point Index'); plt.ylabel('Current (mA)'); plt.grid(True); plt.legend()
        
        plt.tight_layout(rect=[0, 0.03, 1, 0.95]); plt.show()

    Vgrid = np.arange(4.2, 3.2, -0.01)
    Qpdf = np.array([])
    Cycle_count = np.array([])
    Q_limited_list = np.array([])
    maxV = np.array([])
    maxVpos = np.array([])
    
    # Cycle_Number를 기준으로 데이터 그룹화
    for name, group in d1.groupby('Cycle_Number'):
        Vd = group.Voltage_V.values
        Id = -group.Current_mA.values
        time = group.Time_s.values
        
        if len(Vd) > 1 and len(time) > 1:
            time = time - time[0]
            # 전류(mA)와 시간(s)으로 용량(mAh) 계산
            Qd = 1 - (integrate.cumulative_trapezoid(Id, x=time, initial=0) / 3.6 / q_rated)
            
            # Vgrid에 맞춰 용량 보간
            Q = np.flip(np.interp(Vgrid, np.flip(Vd), np.flip(Qd)))
            Qpdf = np.append(Qpdf, Q)
            Q_limited_list = np.append(Q_limited_list, Q[0] - Q[-1])
            
            # dQ/dV 계산 및 피크 값 추출
            dQdV_vals = np.diff(Q)
            if len(dQdV_vals) > 0:
                maxV = np.append(maxV, np.max(dQdV_vals))
                idx = np.argmin(np.abs(dQdV_vals - np.max(dQdV_vals)))
                maxVpos = np.append(maxVpos, Vgrid[idx])
            else: # dQ/dV 계산이 불가능한 경우 0으로 채움
                maxV = np.append(maxV, 0)
                maxVpos = np.append(maxVpos, 0)

            Cycle_count = np.append(Cycle_count, int(name))

    # 초기값을 기준으로 변화량 계산
    if len(maxV) > 0: maxV = maxV - maxV[0]
    if len(Q_limited_list) > 0: Q_limited_list = Q_limited_list - Q_limited_list[0]
    if len(maxVpos) > 0: maxVpos = maxVpos - maxVpos[0]
            
    return maxV, np.asarray(Q_limited_list), Cycle_count


In [ ]:
# Cell 3: Evaluation Metrics
def MAPE(y_test, y_pred):
    return np.mean(np.abs((y_test - y_pred) / y_test)) * 100

def mape_loss(preds, target):
    epsilon = 1e-8
    return torch.mean(torch.abs((preds - target) / (target + epsilon))) * 100

def RMSPE(y_true, y_pred):
    percentage_errors = (y_true - y_pred) / y_true
    squared_percentage_errors = np.square(percentage_errors)
    mean_squared_percentage_error = np.mean(squared_percentage_errors)
    rmspe_value = np.sqrt(mean_squared_percentage_error) * 100
    return rmspe_value


In [ ]:
# Cell 4: Define Dataset Paths and Information

# 데이터 파일이 있는 상위 폴더 경로를 설정해주세요.
# 예: DATA_ROOT = "C:/Users/user/data/"
DATA_ROOT = "../../../data/processed/" # 사용자의 환경에 맞게 수정하세요.

TRAIN_SET = {
    "ch09_SaveData_concatenated_p22_discharge_s3.csv": [os.path.join(DATA_ROOT, "Dataset_A1_profile/A1_MP1_4500mAh_T23/"), 4500],
    "ch10_SaveData_concatenated_p22_discharge_s3.csv": [os.path.join(DATA_ROOT, "Dataset_A1_profile/A1_MP1_4500mAh_T23/"), 4500]
}
TEST_SET = {
    "ch19_SaveData_concatenated_p22_discharge_s3.csv": [os.path.join(DATA_ROOT, "Dataset_A1_profile/A1_MP2_4470mAh_T23/"), 4470],
    "ch06_SaveData_concatenated_p22_discharge_s3.csv": [os.path.join(DATA_ROOT, "Dataset_A1_profile/A1_MP2_4470mAh_T23/"), 4470]
}

print("Train Set Files:")
for f in TRAIN_SET.keys(): print(f"- {f}")
    
print("\nTest Set Files:")
for f in TEST_SET.keys(): print(f"- {f}")


In [ ]:
# Cell 5: Calculate Global Normalization Parameters (using Training Set only)

print("Calculating normalization parameters from the training set...")
# 참고: 데이터 누수를 막기 위해 학습 데이터만으로 정규화 기준을 계산합니다.
for key in tqdm(TRAIN_SET.keys(), desc="Processing Train Set"):
    src = TRAIN_SET[key][0]
    rated_capacity = TRAIN_SET[key][1]
    
    # plot_data=False로 설정하여 계산 중에는 그래프를 그리지 않음
    d_max_dQdV, d_cap_limit, cyc = dQdV(os.path.join(src, key), rated_capacity, plot_data=False)
    
    if len(d_max_dQdV) > 0: maxV_global = max(maxV_global, np.max(np.abs(d_max_dQdV)))
    if len(d_cap_limit) > 0: Q_limited_list_max_global = max(Q_limited_list_max_global, np.max(np.abs(d_cap_limit)))
    # Cycle_count_max_global은 고정값이므로 업데이트하지 않음 (또는 학습 데이터의 최대 사이클로 설정 가능)
    # if len(cyc) > 0: Cycle_count_max_global = max(Cycle_count_max_global, np.max(cyc))
    
print("\n--- Global Normalization Parameters ---")
print(f"maxV_global: {maxV_global}")
print(f"Q_limited_list_max_global: {Q_limited_list_max_global}")
print(f"Cycle_count_max_global: {Cycle_count_max_global}")


In [ ]:
# Cell 6: Prepare Training Data (Feature Extraction & Windowing)

windowsize = 100

X_raw_train = []
y_raw_train = []

print("Preparing training data...")
for key in tqdm(TRAIN_SET.keys(), desc="Creating Train Sequences"):
    src = TRAIN_SET[key][0]
    rated_capacity = TRAIN_SET[key][1]
    
    # 한 번만 그래프를 그려서 데이터 확인 (첫 번째 파일만)
    plot_first_file = (key == list(TRAIN_SET.keys())[0])
    d_max_dQdV, d_cap_limit, cyc = dQdV(os.path.join(src, key), rated_capacity, plot_data=plot_first_file)

    # 데이터가 비어있는 경우 건너뛰기
    if len(cyc) == 0:
        continue
        
    # 정규화 (0으로 나누는 것을 방지)
    d_max_dQdV = d_max_dQdV / maxV_global if maxV_global > 0 else d_max_dQdV
    d_cap_limit = d_cap_limit / Q_limited_list_max_global if Q_limited_list_max_global > 0 else d_cap_limit
    cyc = cyc / Cycle_count_max_global

    # 입력 특징: (dQ/dV 피크, 사이클 수)
    features = np.concatenate((np.expand_dims(d_max_dQdV, axis=-1), np.expand_dims(cyc, axis=-1)), axis=1).tolist()
    
    X_raw_train.extend(features)
    y_raw_train.extend(d_cap_limit.tolist())

# 슬라이딩 윈도우로 시퀀스 데이터 생성
X_train = []
y_train = []
if len(X_raw_train) >= windowsize:
    for i in range(len(X_raw_train) - windowsize + 1):
        X_train.append(X_raw_train[i:i+windowsize])
        y_train.append(y_raw_train[i:i+windowsize])

X_train = np.asarray(X_train)
y_train = np.asarray(y_train)

print("\n--- Training Data Shape ---")
print("X_train.shape:", X_train.shape)
print("y_train.shape:", y_train.shape)


In [ ]:
# Cell 7: Prepare Test Data

X_raw_test_list = []
y_raw_test_list = []
test_cells = []

print("Preparing test data...")
for key in tqdm(TEST_SET.keys(), desc="Creating Test Sequences"):
    src = TEST_SET[key][0]
    rated_capacity = TEST_SET[key][1]
    
    d_max_dQdV, d_cap_limit, cyc = dQdV(os.path.join(src, key), rated_capacity, plot_data=False)

    if len(cyc) == 0:
        continue
        
    # 정규화 (학습 데이터의 파라미터 사용)
    d_max_dQdV = d_max_dQdV / maxV_global if maxV_global > 0 else d_max_dQdV
    d_cap_limit = d_cap_limit / Q_limited_list_max_global if Q_limited_list_max_global > 0 else d_cap_limit
    cyc = cyc / Cycle_count_max_global

    features = np.concatenate((np.expand_dims(d_max_dQdV, axis=-1), np.expand_dims(cyc, axis=-1)), axis=1).tolist()
    y_raw = d_cap_limit.tolist()

    # 슬라이딩 윈도우 적용
    X_test_sub = []
    y_test_sub = []
    if len(features) >= windowsize:
        for i in range(len(features) - windowsize + 1):
            X_test_sub.append(features[i:i+windowsize])
            y_test_sub.append(y_raw[i:i+windowsize])
    
    if X_test_sub: # 데이터가 있는 경우에만 추가
        test_cells.append([np.asarray(X_test_sub), np.asarray(y_test_sub)])

print("\n--- Test Data Shape ---")
for i, (x, y) in enumerate(test_cells):
    print(f"Test Cell {i}: X shape {x.shape}, y shape {y.shape}")


In [ ]:




# Cell 2: Core Feature Extraction Function (dQdV)
def dQdV(file, q_rated, plot_data=False):
    """
    배터리 사이클 데이터 파일로부터 dQ/dV 특징을 추출합니다.
    
    Args:
        file (str): CSV 파일 경로
        q_rated (int): 배터리의 정격 용량 (mAh)
        plot_data (bool): 데이터 확인을 위한 그래프를 그릴지 여부
        
    Returns:
        tuple: (maxV, Q_limited_list, Cycle_count) 시계열 데이터
    """
    # 사용자가 명시한 모든 컬럼을 우선 읽어들입니다.
    # 만약 파일에 특정 컬럼이 없다면 에러가 발생할 수 있습니다.
    try:
        all_cols = ["Cycle_Number","Time_s","Voltage_V","Current_mA","Temperature_C","Capacity_mAh","Power_W","Energy_Wh","AverageVoltage"]
        d1 = pd.read_csv(file, usecols=lambda c: c in all_cols)
    except Exception:
        # 일부 컬럼이 없는 경우를 대비한 예외 처리
        d1 = pd.read_csv(file)[["Cycle_Number", "Voltage_V", "Current_mA", "Time_s", "Capacity_mAh"]]

    
    # 요청하신 데이터 시각화 부분
    if plot_data:
        print(f"--- 데이터 확인: {Path(file).name} ---")
        
        # 1. 전체 데이터의 전압 및 전류 플롯
        plt.figure(figsize=(12, 6))
        plt.suptitle(f'Overall Data for {Path(file).name}', fontsize=16)
        
        plt.subplot(2, 1, 1)
        plt.plot(d1.index, d1['Voltage_V'], label='Voltage (V)')
        plt.title('Voltage over all cycles')
        plt.xlabel('Data Point Index'); plt.ylabel('Voltage (V)'); plt.grid(True); plt.legend()
        
        plt.subplot(2, 1, 2)
        plt.plot(d1.index, d1['Current_mA'], label='Current (mA)', color='orange')
        plt.title('Current over all cycles')
        plt.xlabel('Data Point Index'); plt.ylabel('Current (mA)'); plt.grid(True); plt.legend()
        
        plt.tight_layout(rect=[0, 0.03, 1, 0.95]); plt.show()

        # 2. 【추가된 플롯】 사이클별 Capacity vs. Voltage 플롯
        plt.figure(figsize=(12, 8))
        plt.title(f'Capacity vs. Voltage Profile for {Path(file).name}', fontsize=14)
        
        unique_cycles = d1['Cycle_Number'].unique()
        # 너무 많은 사이클을 그리면 복잡하므로 일부만 선택 (예: 초기, 중기, 후기 사이클)
        if len(unique_cycles) > 10:
            cycles_to_plot = [unique_cycles[0], unique_cycles[len(unique_cycles)//4], unique_cycles[len(unique_cycles)//2], unique_cycles[-1]]
        else:
            cycles_to_plot = unique_cycles

        for cycle_num in cycles_to_plot:
            cycle_data = d1[d1['Cycle_Number'] == cycle_num]
            # 방전 곡선을 그리기 위해 전압 기준으로 정렬하면 더 깔끔하게 보일 수 있습니다.
            cycle_data = cycle_data.sort_values(by='Voltage_V')
            plt.plot(cycle_data['Voltage_V'], cycle_data['Capacity_mAh'], label=f'Cycle {cycle_num}')

        plt.xlabel('Voltage (V)')
        plt.ylabel('Capacity (mAh)')
        plt.grid(True)
        plt.legend()
        plt.show()


    Vgrid = np.arange(4.2, 3.2, -0.01)
    Qpdf = np.array([])
    Cycle_count = np.array([])
    Q_limited_list = np.array([])
    maxV = np.array([])
    maxVpos = np.array([])
    
    # Cycle_Number를 기준으로 데이터 그룹화
    for name, group in d1.groupby('Cycle_Number'):
        Vd = group.Voltage_V.values
        Id = -group.Current_mA.values
        time = group.Time_s.values
        
        if len(Vd) > 1 and len(time) > 1:
            time = time - time[0]
            # 전류(mA)와 시간(s)으로 용량(mAh) 계산 (Ah가 아닌 mAh 기준)
            Qd = 1 - (integrate.cumulative_trapezoid(Id, x=time, initial=0) / 3.6 / q_rated)
            
            # Vgrid에 맞춰 용량 보간
            Q = np.flip(np.interp(Vgrid, np.flip(Vd), np.flip(Qd)))
            Qpdf = np.append(Qpdf, Q)
            Q_limited_list = np.append(Q_limited_list, Q[0] - Q[-1])
            
            # dQ/dV 계산 및 피크 값 추출
            dQdV_vals = np.diff(Q)
            if len(dQdV_vals) > 0:
                maxV = np.append(maxV, np.max(dQdV_vals))
                idx = np.argmin(np.abs(dQdV_vals - np.max(dQdV_vals)))
                maxVpos = np.append(maxVpos, Vgrid[idx])
            else: # dQ/dV 계산이 불가능한 경우 0으로 채움
                maxV = np.append(maxV, 0)
                maxVpos = np.append(maxVpos, 0)

            Cycle_count = np.append(Cycle_count, int(name))

    # 초기값을 기준으로 변화량 계산
    if len(maxV) > 0: maxV = maxV - maxV[0]
    if len(Q_limited_list) > 0: Q_limited_list = Q_limited_list - Q_limited_list[0]
    if len(maxVpos) > 0: maxVpos = maxVpos - maxVpos[0]
            
    return maxV, np.asarray(Q_limited_list), Cycle_count


# Cell 2: Core Feature Extraction Function (dQdV) with Advanced Visualization
def dQdV(file, q_rated, plot_data=False):
    """
    배터리 사이클 데이터 파일로부터 dQ/dV 특징을 추출합니다.
    (전체 데이터 경향성 시각화 기능 추가)
    
    Args:
        file (str): CSV 파일 경로
        q_rated (int): 배터리의 정격 용량 (mAh)
        plot_data (bool): 데이터 확인을 위한 그래프를 그릴지 여부
        
    Returns:
        tuple: (maxV, Q_limited_list, Cycle_count) 시계열 데이터
    """
    try:
        all_cols = ["Cycle_Number","Time_s","Voltage_V","Current_mA","Temperature_C","Capacity_mAh","Power_W","Energy_Wh","AverageVoltage"]
        d1 = pd.read_csv(file, usecols=lambda c: c in all_cols)
    except Exception:
        d1 = pd.read_csv(file)[["Cycle_Number", "Voltage_V", "Current_mA", "Time_s", "Capacity_mAh"]]

    
    if plot_data:
        print(f"--- 데이터 확인: {Path(file).name} ---")
        
        # ======================================================================
        # 방법 1: Colormap을 사용하여 전체 사이클의 경향성 시각화
        # ======================================================================
        plt.figure(figsize=(12, 8))
        plt.title(f'Capacity vs. Voltage Trend with Colormap', fontsize=14)
        
        unique_cycles = d1['Cycle_Number'].unique()
        # 'viridis' 또는 'plasma' 같은 연속적인 색상맵을 선택합니다.
        cmap = plt.get_cmap('viridis') 
        norm = plt.Normalize(vmin=unique_cycles.min(), vmax=unique_cycles.max())

        for cycle_num in unique_cycles:
            cycle_data = d1[d1['Cycle_Number'] == cycle_num].sort_values(by='Voltage_V')
            # 사이클 번호에 따라 색상을 지정합니다.
            color = cmap(norm(cycle_num))
            plt.plot(cycle_data['Voltage_V'], cycle_data['Capacity_mAh'], color=color, alpha=0.4)

        # 오른쪽에 컬러바를 추가하여 색상과 사이클 번호의 관계를 표시합니다.
        sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
        sm.set_array([])
        cbar = plt.colorbar(sm, ax=plt.gca())
        cbar.set_label('Cycle Number')
        
        plt.xlabel('Voltage (V)')
        plt.ylabel('Capacity (mAh)')
        plt.grid(True)
        plt.show()

        # ======================================================================
        # 방법 2: 투명도를 조절하여 데이터 밀도 시각화
        # ======================================================================
        plt.figure(figsize=(12, 8))
        plt.title(f'Capacity vs. Voltage Density with Transparency', fontsize=14)

        for cycle_num in unique_cycles:
            cycle_data = d1[d1['Cycle_Number'] == cycle_num].sort_values(by='Voltage_V')
            # alpha 값을 낮게 설정하여 선을 투명하게 만듭니다.
            plt.plot(cycle_data['Voltage_V'], cycle_data['Capacity_mAh'], color='blue', alpha=0.05)

        plt.xlabel('Voltage (V)')
        plt.ylabel('Capacity (mAh)')
        plt.grid(True)
        plt.show()
        
    # --- 이하 특징 추출 로직은 동일 ---
    Vgrid = np.arange(4.2, 3.2, -0.01)
    Qpdf = np.array([])
    Cycle_count = np.array([])
    Q_limited_list = np.array([])
    maxV = np.array([])
    maxVpos = np.array([])
    
    for name, group in d1.groupby('Cycle_Number'):
        Vd = group.Voltage_V.values
        Id = -group.Current_mA.values
        time = group.Time_s.values
        
        if len(Vd) > 1 and len(time) > 1:
            time = time - time[0]
            Qd = 1 - (integrate.cumulative_trapezoid(Id, x=time, initial=0) / 3.6 / q_rated)
            Q = np.flip(np.interp(Vgrid, np.flip(Vd), np.flip(Qd)))
            Qpdf = np.append(Qpdf, Q)
            Q_limited_list = np.append(Q_limited_list, Q[0] - Q[-1])
            dQdV_vals = np.diff(Q)
            if len(dQdV_vals) > 0:
                maxV = np.append(maxV, np.max(dQdV_vals))
                idx = np.argmin(np.abs(dQdV_vals - np.max(dQdV_vals)))
                maxVpos = np.append(maxVpos, Vgrid[idx])
            else:
                maxV = np.append(maxV, 0)
                maxVpos = np.append(maxVpos, 0)
            Cycle_count = np.append(Cycle_count, int(name))

    if len(maxV) > 0: maxV = maxV - maxV[0]
    if len(Q_limited_list) > 0: Q_limited_list = Q_limited_list - Q_limited_list[0]
    if len(maxVpos) > 0: maxVpos = maxVpos - maxVpos[0]
            
    return maxV, np.asarray(Q_limited_list), Cycle_count


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import integrate
from pathlib import Path

# ======================================================================
# 1. Nature 스타일을 적용하는 헬퍼 함수 정의
# ======================================================================
def apply_nature_style(ax, title, xlabel, ylabel, panel_label=''):
    """
    Matplotlib의 Axes 객체에 Nature 스타일을 적용합니다.

    Args:
        ax (matplotlib.axes.Axes): 스타일을 적용할 Axes 객체
        title (str): 그래프 제목
        xlabel (str): X축 라벨
        ylabel (str): Y축 라벨
        panel_label (str): 패널 라벨 (예: 'a', 'b', 'c')
    """
    # 축 라벨 및 제목 설정
    ax.set_title(title, fontsize=16, fontweight='bold', pad=15)
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)

    # 오른쪽과 위쪽 축(Spine) 제거
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    
    # 축 선 두께 설정
    ax.spines['left'].set_linewidth(1.5)
    ax.spines['bottom'].set_linewidth(1.5)

    # 축 눈금(Ticks)을 안쪽으로 설정하고, 눈금 선 두께 조절
    ax.tick_params(axis='both', which='major', direction='in', length=6, width=1.5)
    
    # 패널 라벨 추가 (굵은 글씨)
    if panel_label:
        ax.text(-0.1, 1.1, panel_label, transform=ax.transAxes, 
                fontsize=18, fontweight='bold', va='top')

    # 그리드 제거
    ax.grid(False)


# ======================================================================
# 2. 핵심 로직 함수 (dQdV) 수정
# ======================================================================
def dQdV(file, q_rated, plot_data=False):
    """
    배터리 사이클 데이터로부터 dQ/dV 특징을 추출하고, 
    Nature 스타일에 맞는 고품질 분석 그래프 3종을 단일 Figure에 생성합니다.
    """
    try:
        all_cols = ["Cycle_Number","Time_s","Voltage_V","Current_mA","Temperature_C","Capacity_mAh","Power_W","Energy_Wh","AverageVoltage"]
        d1 = pd.read_csv(file, usecols=lambda c: c in all_cols)
    except Exception:
        d1 = pd.read_csv(file)[["Cycle_Number", "Voltage_V", "Current_mA", "Time_s", "Capacity_mAh"]]

    
    # 그래프 시각화 로직
    if plot_data:
        # --- 전역 스타일 설정 (함수 호출 시 한 번만 실행) ---
        plt.rcParams.update({
            'figure.dpi': 150,
            'font.family': 'sans-serif',
            'font.sans-serif': ['Arial', 'Helvetica'], # Nature에서 선호하는 산세리프 폰트
            'font.size': 12,
            'axes.labelsize': 14,
            'xtick.labelsize': 12,
            'ytick.labelsize': 12,
            'legend.fontsize': 12,
            'lines.linewidth': 2,
        })

        print(f"--- 데이터 시각화 분석 시작: {Path(file).name} ---")
        
        unique_cycles = d1['Cycle_Number'].unique()
        cmap = plt.get_cmap('viridis')
        norm = plt.Normalize(vmin=unique_cycles.min(), vmax=unique_cycles.max())

        # 데이터 준비 (기존 코드와 동일)
        vq_plot_data = []
        dqdv_unrestricted_plot_data = []
        dqdv_restricted_plot_data = []

        for cycle_num in unique_cycles:
            group = d1[d1['Cycle_Number'] == cycle_num].copy()
            group.sort_values(by='Time_s', inplace=True)
            Vd, Id, time, Q_raw = group.Voltage_V.values, -group.Current_mA.values, group.Time_s.values, group['Capacity_mAh'].values
            if len(Vd) < 2: continue
            vq_plot_data.append({'cycle': cycle_num, 'V': Vd, 'Q': Q_raw})
            time = time - time[0]
            Qd = integrate.cumulative_trapezoid(Id, x=time, initial=0) / 3.6
            valid_indices = np.where(np.abs(np.diff(Vd)) > 1e-5)[0]
            if len(valid_indices) > 1:
                Vd_unres = (Vd[1:] + Vd[:-1])[valid_indices] / 2
                dQdV_unres = np.diff(Qd)[valid_indices] / np.diff(Vd)[valid_indices]
                dqdv_unrestricted_plot_data.append({'cycle': cycle_num, 'V': Vd_unres, 'dQdV': dQdV_unres})
            Vgrid = np.arange(4.2, 3.2, -0.01)
            Q_interp = np.flip(np.interp(Vgrid, np.flip(Vd), np.flip(Qd)))
            V_res = (Vgrid[:-1] + Vgrid[1:]) / 2
            dQdV_res = np.diff(Q_interp) / np.diff(Vgrid)
            dqdv_restricted_plot_data.append({'cycle': cycle_num, 'V': V_res, 'dQdV': dQdV_res})

        # --- 3개의 서브플롯을 가진 단일 Figure 생성 ---
        fig, axes = plt.subplots(3, 1, figsize=(8, 18))
        
        # Plot 1: V-Q Curve
        ax1 = axes[0]
        for data in vq_plot_data:
            ax1.plot(data['Q'], data['V'], color=cmap(norm(data['cycle'])), alpha=0.6)
        apply_nature_style(ax1, 'V-Q Curve', 'Capacity (mAh)', 'Voltage (V)', 'a')

        # Plot 2: Unrestricted dQ/dV
        ax2 = axes[1]
        for data in dqdv_unrestricted_plot_data:
            ax2.plot(data['V'], data['dQdV'], color=cmap(norm(data['cycle'])), alpha=0.6)
        apply_nature_style(ax2, 'Unrestricted dQ/dV vs. Voltage', 'Voltage (V)', 'dQ/dV (mAh/V)', 'b')
        if dqdv_unrestricted_plot_data:
             ax2.set_ylim(0, np.percentile([d['dQdV'].max() for d in dqdv_unrestricted_plot_data if len(d['dQdV']) > 0], 98))

        # Plot 3: Vgrid-limited dQ/dV
        ax3 = axes[2]
        for data in dqdv_restricted_plot_data:
            ax3.plot(data['V'], data['dQdV'], color=cmap(norm(data['cycle'])), alpha=0.6)
        apply_nature_style(ax3, 'Vgrid-limited dQ/dV vs. Voltage', 'Voltage (V)', 'dQ/dV (mAh/V)', 'c')

        # --- Figure 전체에 대한 Colorbar 추가 ---
        sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
        # fig.add_axes()를 사용하여 colorbar 위치를 세밀하게 조정
        cbar_ax = fig.add_axes([0.93, 0.15, 0.03, 0.7]) # [left, bottom, width, height]
        cbar = fig.colorbar(sm, cax=cbar_ax)
        cbar.set_label('Cycle Number', rotation=270, labelpad=20)
        cbar.outline.set_linewidth(1.5)

        # 전체 레이아웃 조정
        fig.tight_layout(rect=[0, 0, 0.9, 1]) # Colorbar와 겹치지 않도록 레이아웃 조정
        
        plt.show()

    # --- 이하 특징 추출 로직은 동일 ---
    Vgrid = np.arange(4.2, 3.2, -0.01)
    # ... (기존 특징 추출 코드) ...
            
    return np.array([]), np.array([]), np.array([]) # 임시 반환